In [7]:
import numpy as np

# 1. Load the data from the text file.
loaded_image_2d = np.loadtxt('image.txt', dtype=np.uint8)

print("--- Loaded 2D data ---")
print("Shape:", loaded_image_2d.shape) 

# 2. Define the CORRECT original dimensions of your image.
original_height = 181  # <-- CORRECTED based on your file
original_width = 279   # <-- CORRECTED based on your file
num_channels = 3       # <-- This is 3 because it's an RGB image

# 3. Reshape the 2D array back into its original 3D shape.
# This should now work without an error.
loaded_image_3d = loaded_image_2d.reshape(original_height, original_width, num_channels)

print("\n--- Reshaped 3D data ---")
print("Shape:", loaded_image_3d.shape)

--- Loaded 2D data ---
Shape: (181, 837)

--- Reshaped 3D data ---
Shape: (181, 279, 3)
